In [1]:
import sys
import os
sys.path.append('../')
import numpy as np
import scipy.signal as sps
from source.reader import read_raw
from source.interference_reject import InterferenceReject
import plotly  as py
import plotly.graph_objects as go

In [22]:
# Data import
file_name = "/home/hri/Hyperfine/HFData/fid/inr_signal.h5"
data, index, metadata = read_raw(file_name)
num_channels_all = metadata['console']['adc']['number_channels']
num_channels_signal = metadata['rf']['receive']['num_signal_channels']
matrix_size = metadata['sequence']['encoding']['matrix_size']
acq_bandwidth = metadata['sequence']['readout']['readouts'][0]['bandwidth']

In [23]:
# correct prescan data
select_signal_channels=[8,9,10,11]
interf_rej_1 = InterferenceReject(channels_signal=select_signal_channels,channels_noise=[12])
interf_rej_2 = InterferenceReject(channels_signal=select_signal_channels,channels_noise=[12,13])
interf_rej_3 = InterferenceReject(channels_signal=select_signal_channels,channels_noise=[12,13,14])
interf_rej_4 = InterferenceReject(channels_signal=select_signal_channels,channels_noise=[12,13,14,15])
data_raw = data['noise']
data_cor_1 = interf_rej_1.apply(data_signal=data_raw, data_noise=data_raw)
data_cor_2 = interf_rej_2.apply(data_signal=data_raw, data_noise=data_raw)
data_cor_3 = interf_rej_3.apply(data_signal=data_raw, data_noise=data_raw)
data_cor_4 = interf_rej_4.apply(data_signal=data_raw, data_noise=data_raw)

## Plot noise density (prescan)

In [24]:
from source.utils import compute_spectrogram
axis_pow, pow_raw = compute_spectrogram(data_raw[select_signal_channels, :, :], acq_bandwidth)
axis_pow, pow_cor_1 = compute_spectrogram(data_cor_1, acq_bandwidth)
axis_pow, pow_cor_2 = compute_spectrogram(data_cor_2, acq_bandwidth)
axis_pow, pow_cor_3 = compute_spectrogram(data_cor_3, acq_bandwidth)
axis_pow, pow_cor_4 = compute_spectrogram(data_cor_4, acq_bandwidth)

ch = 0

# average noise in 10 kHz bandwidth
sel_passband = np.abs(axis_pow) < 5e3
sel_stopband = np.abs(axis_pow) > 15e3
avg_pow_raw = 10 * np.log10(np.mean(pow_raw[ch, sel_passband]))
print("Average raw = {} dB".format(avg_pow_raw))
avg_pow_floor = 10 * np.log10(np.mean(pow_raw[ch, sel_stopband]))
print("Average baseline = {} dB".format(avg_pow_floor))
avg_pow_1ch = 10 * np.log10(np.mean(pow_cor_1[ch, sel_passband]))
print("Average 1 channel = {} dB".format(avg_pow_1ch))
avg_pow_2ch = 10 * np.log10(np.mean(pow_cor_2[ch, sel_passband]))
print("Average 2 channel = {} dB".format(avg_pow_2ch))
avg_pow_3ch = 10 * np.log10(np.mean(pow_cor_3[ch, sel_passband]))
print("Average 2 channel = {} dB".format(avg_pow_3ch))
avg_pow_4ch = 10 * np.log10(np.mean(pow_cor_4[ch, sel_passband]))
print("Average 2 channel = {} dB".format(avg_pow_4ch))

# Interactive plot
fig = go.Figure()
fig.update_layout(plot_bgcolor="white")
line_style = {'color': 'orange', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_raw[ch, :]), line=line_style, name='Raw'))
line_style = {'color': 'gray', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_cor_1[ch, :]), line=line_style, name='1 reference coil'))
line_style = {'color': 'green', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_cor_2[ch, :]), line=line_style, name='2 reference coils'))
line_style = {'color': 'blue', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_cor_3[ch, :]), line=line_style, name='3 reference coils'))
line_style = {'color': 'black', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_cor_4[ch, :]), line=line_style, name='4 reference coils'))
fig.show()


Average raw = -5.890070205541399 dB
Average baseline = -35.88978372847285 dB
Average 1 channel = -13.070719849198413 dB
Average 2 channel = -14.950445875145647 dB
Average 2 channel = -16.31496392147063 dB
Average 2 channel = -17.116412921762496 dB


In [33]:
# Compare with theory sqrt(1/(1+k*INR**2))

# Model: noise coil INR is 4.5 dB, or INR = 1.7 (in amplitude)
k = np.arange(1, 9)
residual_model = 1 / np.sqrt(1 + k * 1.85 ** 2)

# Measured data: ration between raw and corrected
residual_measured = np.zeros(4)
residual_measured[0] = avg_pow_1ch - avg_pow_raw
residual_measured[1] = avg_pow_2ch - avg_pow_raw
residual_measured[2] = avg_pow_3ch - avg_pow_raw
residual_measured[3] = avg_pow_4ch - avg_pow_raw
k_3 = [1, 2, 3, 4]

fig = go.Figure()
fig.update_layout(plot_bgcolor="white", title="Residual Interference", xaxis_title="Reference channels", yaxis_title="residual (dB)")
line_style = {'color': 'black', 'width': 2}
fig.add_trace(go.Scatter(x=k, y=20 * np.log10(residual_model), line=line_style, name='Model', mode='lines'))
line_style = {'color': 'blue', 'width': 2}
fig.add_trace(go.Scatter(x=k_3, y=residual_measured, line=line_style, name='Measured', mode='markers'))
fig.show()